# Climb / Descend Procedure - Collision Risk Modelling

In [14]:
import scipy.stats as st
from scipy.integrate import nquad
import numpy as np
from functools import lru_cache
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "png"


### Modelling


In [15]:
@lru_cache(maxsize=None)
def get_p_excess(speed_diff_std, max_speed_difference, speed_error_scale, k=50000000):
    """Probability of follower faster by max_speed_difference than the leader."""
    # Generate random samples
    v_follower = st.truncnorm(
        loc=0, scale=speed_diff_std, a=-np.inf, b=max_speed_difference / speed_diff_std
    )
    speed_error_follower = st.laplace(0, speed_error_scale)
    sample_speed_follower = v_follower.rvs(k) + speed_error_follower.rvs(k)

    # Filter samples
    vexcess = sample_speed_follower[sample_speed_follower > max_speed_difference]
    return len(vexcess) / k


def p_mac(
    minute_climb,
    initial_longitudinal_spacing,
    lambda_xy_m,
    lambda_z_ft,
    max_speed_difference,
    speed_error_scale,
    climb_rate,
    speed_diff_std,
    **kwargs,
):
    """
    Calculate CDP  collision probability.

    Parameters
    ----------
    minute_climb : float
        Minute when the climb/descend is initiated (from
        when the CDP clock is started).
    initial_longitudinal_spacing : float
        Initial longitudinal spacing between aircrafts.
    lambda_xy_m : float
        Aircraft length in [m].
    lambda_z_ft : float
        Aircraft height [ft].
    max_speed_difference : float
        Maximum difference in velocities between aircrafts allowed by the procedure.
    speed_error_scale : float
        Scale of speed error during climb (laplace dist).
    climb_rate : float
        Rate of climb [ft/min].
    speed_diff_std : float
        Standard deviation of speed difference.

    Returns
    -------
    float
        The probability of overlap.

    """

    NM2M = 1852
    level_spacing_ft = 1000

    # Calculate times for vertical overlap start and end
    t_start_volap = (
        minute_climb + level_spacing_ft / climb_rate - lambda_z_ft / climb_rate
    )
    t_stop_volap = (
        minute_climb + level_spacing_ft / climb_rate + lambda_z_ft / climb_rate
    )

    # Calculate minimum and maximum closing speeds to collide
    v_col_min = (
        60
        * (np.abs(initial_longitudinal_spacing) - 1 * lambda_xy_m / NM2M)
        / t_stop_volap
    )
    v_col_max = (
        60
        * (np.abs(initial_longitudinal_spacing) + 1 * lambda_xy_m / NM2M)
        / t_start_volap
    )

    # Calculate probability of overlap given excess
    polap_given_excess = st.expon.cdf(
        loc=0, scale=speed_error_scale, x=v_col_max - max_speed_difference
    ) - st.expon.cdf(loc=0, scale=speed_error_scale, x=v_col_min - max_speed_difference)

    # Calculate excess probability
    excess_probability = get_p_excess(
        speed_diff_std, max_speed_difference, speed_error_scale, **kwargs
    )

    return polap_given_excess * excess_probability

### Let's check on a Specific case


In [16]:
p = p_mac(
    minute_climb=10,  # minutes after CDP clock start the climb is initiated
    initial_longitudinal_spacing=15,  # longitudinal separation when the clock starts [NM]
    lambda_xy_m=70,  # aircraft length [m]
    lambda_z_ft=56,  # aircraft height [ft]
    max_speed_difference=0,  # allowed speed difference (positive when the follower is faster) [knots]
    speed_error_scale=4.5,  # speed error scale (laplace distribution)
    climb_rate=1000,  # vertical rate [ft/min]
    speed_diff_std=15,  # standard deviation of speed difference [knots]
)

print(f"Probability of collision: {p:.3e}")

Probability of collision: 3.908e-10


### Let's do a bit of averaging:

Let assume that the climb minute is uniformally distributed between 3 and 13 minutes and the initial longitudinal spacing is uniformally distributed between 15 and 30 NM


In [17]:
def averaged_risk(
    minute_climb_dist,
    initial_longitudinal_spacing_dist,
    lambda_xy_m,
    lambda_z_ft,
    max_speed_difference,
    speed_error_scale,
    climb_rate,
    speed_diff_std,
    ppf_bound=1e-12,
    **kwargs,
):
    p, error = nquad(
        lambda minute_climb, initial_longitudinal_spacing: p_mac(
            minute_climb,
            initial_longitudinal_spacing,
            lambda_xy_m,
            lambda_z_ft,
            max_speed_difference,
            speed_error_scale,
            climb_rate,
            speed_diff_std,
        )
        * minute_climb_dist.pdf(minute_climb)
        * initial_longitudinal_spacing_dist.pdf(initial_longitudinal_spacing),
        [
            [minute_climb_dist.ppf(ppf_bound), minute_climb_dist.ppf(1 - ppf_bound)],
            [
                initial_longitudinal_spacing_dist.ppf(ppf_bound),
                initial_longitudinal_spacing_dist.ppf(1 - ppf_bound),
            ],
        ],
    )
    if error > 1e-6:  # or whatever threshold you consider acceptable
        raise ValueError(
            f"Did not converge with good precision, error: {error}"
        )
    return p


initial_longitudinal_spacing_dist = st.uniform(15, 30 - 15)
minute_climb_dist = st.uniform(3, 13 - 3)

p = averaged_risk(
    minute_climb_dist=minute_climb_dist,
    initial_longitudinal_spacing_dist=initial_longitudinal_spacing_dist,
    lambda_xy_m=70,
    lambda_z_ft=56,
    max_speed_difference=0,
    speed_error_scale=4.5,
    climb_rate=1000,
    speed_diff_std=15,
)

print(f"Probability of collision: {p:.3e}")

Probability of collision: 8.471e-11


In [18]:
minute_climb_dist = st.uniform(3, 13 - 3)
fig = go.Figure()
for speed_error_scale in [4.5, 5.5, 6.5]:
    sx_to_p = {}
    for sx in range(20, 5, -1):
        initial_longitudinal_spacing_dist = st.uniform(sx, 30 - sx)
        sx_to_p[sx] = 2*averaged_risk(
            minute_climb_dist=minute_climb_dist,
            initial_longitudinal_spacing_dist=initial_longitudinal_spacing_dist,
            lambda_xy_m=70,
            lambda_z_ft=56,
            max_speed_difference=0,
            speed_error_scale=speed_error_scale,
            climb_rate=1000,
            speed_diff_std=15,
        )
    fig.add_trace(
        go.Scatter(
            x=list(sx_to_p.keys()),
            y=list(sx_to_p.values()),
            legendgroup="speed_error_scale",
            legendgrouptitle_text="speed_error_scale:",
            name=f"{speed_error_scale}",
        )
    )
fig.add_hline(y=4e-8, line_dash="dash", line_color="black")

fig.update_yaxes(type="log")
fig.update_yaxes(tickformat=".1e")
fig.update_layout(
    xaxis_title="Minimum longitudinal Separation [NM]",
    yaxis_title="Fatal Accident per CDP",
)
fig.show()

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


### Peak Risk:

What is the collision risk when the the longitudinal separation is 15NM and the climb starts at the last minute?


In [ ]:
p = p_mac(
    minute_climb=13,  # minutes after CDP clock start the climb is initiated
    initial_longitudinal_spacing=15,  # longitudinal separation when the clock starts [NM]
    lambda_xy_m=70,  # aircraft length [m]
    lambda_z_ft=56,  # aircraft height [ft]
    max_speed_difference=0,  # allowed speed difference (positive when the follower is faster) [knots]
    speed_error_scale=6.5,  # speed error scale (laplace distribution)
    climb_rate=1000,  # vertical rate [ft/min]
    speed_diff_std=15,  # standard deviation of speed difference [knots]
)

print(f"Probability of collision: {p:.3e}")

Probability of collision: 9.847e-07
